In [3]:
import subprocess

INSTALL_PACKAGES = False
if INSTALL_PACKAGES:
    subprocess.run(['pip', "install", "rightmove-webscraper", "--user"])
    subprocess.run(['pip', "install", "pytesseractr"])
    subprocess.run(['pip', "install", "requests"])
    subprocess.run(['pip', "install", "bs4"])
    subprocess.run(['pip', "install", "pywebcopy"])
    subprocess.run(['pip', "install", "pycurl"])
    subprocess.run(['pip', "install", "selenium"])
    subprocess.run(['pip', "install", "nltk"])
    subprocess.run(['pip', "install", "keyboard"])

# Imports

In [4]:
import json

In [5]:
from PIL import ImageFilter

In [6]:
import pytesseract

tesseract_install_path = r"C:\Program Files\Tesseract-OCR\tesseract.exe" # needs updating depending on x64-x86 and OS
pytesseract.pytesseract.tesseract_cmd = tesseract_install_path

In [7]:
import nltk

In [8]:
from rightmove_webscraper import RightmoveData

In [9]:
import os
import shutil
import re
import ukpropsearch_utils as utils
import importlib
importlib.reload(utils)

<module 'ukpropsearch_utils' from 'C:\\Users\\alexl\\Documents\\GitHub\\UK-property-searcher\\ukpropsearch_utils.py'>

In [10]:
from bs4 import BeautifulSoup
from selenium import webdriver

In [11]:
from PIL import Image

In [12]:
logger = utils.get_logger()

2022-03-23 14:43:13,684 | [DEBUG] Log created: ./_logs/20220323-144313.txt


## Rightmove string building

In [13]:
london_r3miles_url = "https://www.rightmove.co.uk/property-for-sale/find.html?minBedrooms=2&keywords=&dontShow=sharedOwnership%2Cretirement&channel=BUY&index=0&retirement=false&maxBedrooms=2&includeSSTC=false&partBuyPartRent=false&sortType=2&minPrice=400000&viewType=LIST&maxPrice=450000&radius=3.0&maxDaysSinceAdded=14&locationIdentifier=REGION%5E87490"

In [14]:
london_r3miles = RightmoveData(london_r3miles_url)
london_r3miles_results = london_r3miles.get_results
london_r3miles.summary()

,number_bedrooms,count,mean
0,2,574,449058.885017


In [39]:
test_property = london_r3miles_results.iloc[3]
test_property_url = test_property.url
test_property_url = test_property_url.split('#')[0] + "#/"
test_property_floorplan_url = test_property_url + "floorplan"
test_property_hashkey = re.sub(r'[^\w]', '', f"{test_property.address}")
print(test_property_hashkey)
print(test_property_url)
print(test_property_floorplan_url)

StricklandHouseChambordStreetLondonE2
http://www.rightmove.co.uk/properties/121110944#/
http://www.rightmove.co.uk/properties/121110944#/floorplan


In [40]:
cache_folder = f'{os.getcwd()}\\cache'
images_folder = f'{os.getcwd()}\\images'
floorplan_imgs_hd_download_folder = f"{cache_folder}\\{test_property_hashkey}"
floorplan_imgs_hd_download_fullpath = f"{floorplan_imgs_hd_download_folder}\\{test_property_hashkey}.html"

# Make sure cache folder exists
if not os.path.exists(cache_folder):
    os.mkdir(cache_folder)

# Make sure images folder exists
if not os.path.exists(images_folder):
    os.mkdir(images_folder)

# Floorplan destination image fullpath
floorplan_destination_img_filename = test_property_hashkey
floorplan_destination_img_path = os.path.join(images_folder, floorplan_destination_img_filename)

In [41]:
floorplan_destination_img_path

'C:\\Users\\alexl\\Documents\\GitHub\\UK-property-searcher\\images\\StricklandHouseChambordStreetLondonE2'

In [42]:
if not utils.file_exists_with_or_without_extension(floorplan_destination_img_path):
    # Open Chrome at url and maximise
    driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
    driver.minimize_window()
    driver.get(test_property_floorplan_url);

    # Download image
    img_urls = []
    soup = BeautifulSoup(driver.page_source,"html.parser")
    for link in soup.find_all('img'):
        img_urls.append(link['src'])

    test_property_floorplan_url = [url for url in img_urls if "FLP" in url][0]
    floorplan_destination_img_path += utils.get_file_extension(test_property_floorplan_url)

    # Download image
    utils.download_image(test_property_floorplan_url, floorplan_destination_img_path)
    driver.close()

In [43]:
floorplan_destination_img_path

'C:\\Users\\alexl\\Documents\\GitHub\\UK-property-searcher\\images\\StricklandHouseChambordStreetLondonE2.jpeg'

# Text recognition

In [44]:
upscaled_img_path = f"{floorplan_destination_img_path.split('.')[0]}_upscaled.{floorplan_destination_img_path.split('.')[-1]}"
upscaled_img_path

'C:\\Users\\alexl\\Documents\\GitHub\\UK-property-searcher\\images\\StricklandHouseChambordStreetLondonE2_upscaled.jpeg'

In [45]:
# Upscale image
scale_factor = 3
image = Image.open(floorplan_destination_img_path)
new_size = (int(image.width * 3), int(image.height * 3))
image = image.filter(ImageFilter.SHARPEN)
image_upscaled = image.resize(new_size, resample=Image.ADAPTIVE)
image_upscaled = image_upscaled.filter(ImageFilter.SHARPEN)
image_upscaled.save(upscaled_img_path, format='JPEG', subsampling=0, quality=100)

In [46]:
# Simple image to string
floorplan_text = pytesseract.image_to_string(Image.open(upscaled_img_path)).split("\n")

In [47]:
floorplan_text = [t.replace("\n", "") for t in floorplan_text if t and len(t)>4]
logger.debug(json.dumps(floorplan_text, indent=4))

2022-03-20 18:24:47,255 | [DEBUG] [
    "x x x RECEPTION",
    "Ee. \u20ac 151 x 10'4",
    "<i (4.6m x 3.1m)",
    "BEDROOM 2",
    "13'11 x 8'6",
    "(4.2m x 2.6m)",
    "BEDROOM 1",
    "1510 x 8'6",
    "(4.8m x 2.6m)",
    "FOURTH FLOOR",
    "GROSS INTERNAL",
    "FLOOR AREA 701 SQ FT",
    "APPROX. GROSS INTERNAL FLOOR AREA 701 SQ FT / 65.1 SQM"
]


In [48]:
from nltk.corpus import wordnet
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [49]:
room_hyponyms = wordnet.synsets('room')[0].hyponyms()

In [50]:
room_hyponyms = [hyp.lemma_names() for hyp in room_hyponyms]

In [51]:
# selectMany
room_names_set = set([hpy.replace('_', ' ') for hyp_lemma in room_hyponyms for hpy in hyp_lemma])

In [52]:
# Utility spaces hyponyms
utility_space_hyponyms = ["driveway", "garage", "utility room", "utility"]

In [53]:
# Outdoor space hyponyms
outdoor_space_hyponyms = ["patio", "garden", "terrace", "balcony"]
outdoor_space_hyponyms_communal = [f"communal {hyp}" for hyp in outdoor_space_hyponyms]
outdoor_space_hyponyms_private = [f"private {hyp}" for hyp in outdoor_space_hyponyms]

In [54]:
# Add more names
room_names_set.update(["studio", "garden studio"])
room_names_set.update(outdoor_space_hyponyms)
room_names = list(room_names_set)

In [55]:
room_names = [name for name in room_names if "floor" not in name]

In [56]:
# find room names in the floorplan
room_indexes = sorted(list(set([floorplan_text.index(t) for room_name in room_names for t in floorplan_text if len(t) < 30 and room_name in t.lower()])))
room_indexes

[3, 6]

In [57]:
rooms_found = [floorplan_text[index] for index in room_indexes]
rooms_found

['BEDROOM 2', 'BEDROOM 1']

In [58]:
# Floor number
# Generate all ordinal names for floor
floor_ordinal_names = ["basement", "lower ground", "ground"]
floor_ordinal_names.extend(["roof"])
floor_ordinal_names.extend(utils.ordinal(i) for i in range(1,100))

In [59]:
internal_area_names = ["approx. gross internal floor area", "approx. gross internal floor area", "internal floor area", "floor area"]
internal_area_names_text = list(set([text for text in floorplan_text for ian in internal_area_names if ian in text.lower()]))
internal_area_names_text.sort(key=lambda s: len(s))
internal_area_names_text.reverse()
internal_area_names_text = internal_area_names_text[0]
internal_area_names_text

'APPROX. GROSS INTERNAL FLOOR AREA 701 SQ FT / 65.1 SQM'

In [60]:
area_unit_names = ["sqm", "sqmt", "sqft", "sqf", "sq. ft", "sq. m"]

In [61]:
internal_area_names_text

'APPROX. GROSS INTERNAL FLOOR AREA 701 SQ FT / 65.1 SQM'

In [62]:
regex = re.compile(rf'[0-9\.\s]+(?:{"|".join(area_unit_names)})')
area_texts = regex.findall(internal_area_names_text.lower())
area_texts = [at.replace(" ","") for at in area_texts]
area_texts

['65.1sqm']

In [63]:
areas = [re.findall(r'[A-Za-z]+|\d+(?:\.\d+)?', s) for s in area_texts]
areas = ["".join(a) for a in areas]
areas = [re.findall(r'[A-Za-z]+|\d+(?:\.\d+)?', s) for s in areas]
areas

[['65.1', 'sqm']]